In [14]:
import copy
import json
from aligner_word import Aligner
import re
import string
from camel_tools.utils.charsets import UNICODE_PUNCT_SYMBOL_CHARSET
from camel_tools.utils.normalize import (
    normalize_alef_ar,
    normalize_alef_maksura_ar,
    normalize_teh_marbuta_ar
)
import editdistance
PUNCS = string.punctuation + ''.join(list(UNICODE_PUNCT_SYMBOL_CHARSET))

In [1]:
x = "ألتحق بكلية الشريعة أنا طالب في جامعة الإمام محمد بن سعود الاسلامية الآن أدرس في تعليم اللغة العربية المستوى الرابع . وهذا الفصل آخر الفصل في المعهد . انتهى هذا الفصل ندخل إلى دبلوم يحتاج إلى سنة كاملة . أي بعد سنة واحدة ندخل الكلية . إن شاء الله سأختار إلى كلية الشريعة لأسباب كثيرة وأتخلص ما يلي : 1 - كلية الشريعة كاملة وشاملة تشمل الموضوعات كثيرة مثل : الفرائض والفقه وأصول الفقه وقواعد اللغة العربية والتوحيد والتفسير إلخ . وهذه المعلومات تدرس في كلية الشريعة إن شاء الله سأستفيد منها كثيرا 2 - وضع الدولة : من المعلوم أن الصين دولة الكفار كثير من المسلمين يتأثر ببوذية كثيرا . ووضعوا قبور الأولياء والصالحين بناء جميلة ليدعو إليه ويوف إليه تبرك به ويستعان به وهذه الخرافات والبدع أزيلها بالطريق الصحيح . وهذه الطرق الصحيح ندرس في الكلية . 3 - السبب النفسية : من بداية دراسة اللغة العربية والدينية أنا أحب مواد الشريعة من مواد أخرى . دائما أقرأ كتب الشرعية ."
y = "ألتحق بكلية الشريعة أنا طالب في جامعة الإمام محمد بن سعود الاسلامية الآن أدرس في تعليم اللغة العربية المستوى الرابع . وهذا الفصل آخر الفصل في المعهد . انتهى هذا الفصل ندخل إلى دبلوم يحتاج إلى سنة كاملة . أي بعد سنة واحدة ندخل الكلية . إن شاء الله سأختار إلى كلية الشريعة لأسباب كثيرة وأتخلص ما يلي : 1 - كلية الشريعة كاملة وشاملة تشمل الموضوعات كثيرة مثل : الفرائض والفقه وأصول الفقه وقواعد اللغة العربية والتوحيد والتفسير إلخ . وهذه المعلومات تدرس في كلية الشريعة إن شاء الله سأستفيد منها كثيرا 2 - وضع الدولة : من المعلوم أن الصين دولة الكفار كثير من المسلمين يتأثر ببوذية كثيرا . ووضعوا قبور الأولياء والصالحين بناء جميلة ليدعو إليه ويوف إليه تبرك به ويستعان به وهذه الخرافات والبدع أزيلها بالطريق الصحيح . وهذه الطرق الصحيح ندرس في الكلية .   3 - السبب النفسية : من بداية دراسة اللغة العربية والدينية أنا أحب مواد الشريعة من مواد أخرى . دائما أقرأ كتب الشرعية ."

In [16]:
y.split() == x.split()

True

In [18]:
len(y)

865

In [17]:
len(x)

863

In [14]:
print(x.split())

['ألتحق', 'بكلية', 'الشريعة', 'أنا', 'طالب', 'في', 'جامعة', 'الإمام', 'محمد', 'بن', 'سعود', 'الاسلامية', 'الآن', 'أدرس', 'في', 'تعليم', 'اللغة', 'العربية', 'المستوى', 'الرابع', '.', 'وهذا', 'الفصل', 'آخر', 'الفصل', 'في', 'المعهد', '.', 'انتهى', 'هذا', 'الفصل', 'ندخل', 'إلى', 'دبلوم', 'يحتاج', 'إلى', 'سنة', 'كاملة', '.', 'أي', 'بعد', 'سنة', 'واحدة', 'ندخل', 'الكلية', '.', 'إن', 'شاء', 'الله', 'سأختار', 'إلى', 'كلية', 'الشريعة', 'لأسباب', 'كثيرة', 'وأتخلص', 'ما', 'يلي', ':', '1', '-', 'كلية', 'الشريعة', 'كاملة', 'وشاملة', 'تشمل', 'الموضوعات', 'كثيرة', 'مثل', ':', 'الفرائض', 'والفقه', 'وأصول', 'الفقه', 'وقواعد', 'اللغة', 'العربية', 'والتوحيد', 'والتفسير', 'إلخ', '.', 'وهذه', 'المعلومات', 'تدرس', 'في', 'كلية', 'الشريعة', 'إن', 'شاء', 'الله', 'سأستفيد', 'منها', 'كثيرا', '2', '-', 'وضع', 'الدولة', ':', 'من', 'المعلوم', 'أن', 'الصين', 'دولة', 'الكفار', 'كثير', 'من', 'المسلمين', 'يتأثر', 'ببوذية', 'كثيرا', '.', 'ووضعوا', 'قبور', 'الأولياء', 'والصالحين', 'بناء', 'جميلة', 'ليدعو', 'إليه', 'ويوف'

In [9]:
x == y

False

In [3]:
len(x.split())

163

In [4]:
len(y.split())

163

In [15]:
def read_data(path):
    with open(path) as f:
        return [x.strip() for x in f.readlines()]

In [16]:
def read_alignment(path):
    example = []
    examples = []
    with open(path, mode='r') as f:
        for line in f.readlines():
            line = line.strip()
            if line:
                data = line.split('\t')
                ex = data[:-1] + [eval(data[-1])]
                example.append(ex)
            else:
                examples.append(example)
                example = []
        
        # adding the last example
        if example:
            examples.append(example)
    return examples

In [17]:
class Range:
    def __init__(self, start, end, ops):
        self.start = start
        self.end = end
        self.ops = ops
        
    def __repr__(self):
        return str(self.to_dict())

    def to_json_str(self):
        return json.dumps(self.to_dict(), indent=2, ensure_ascii=False)

    def to_dict(self):
        output = copy.deepcopy(self.__dict__)
        return output
    
    
def sort_range(v):
    span = v[-1]
    return [span[0], span[0]] if span[1] == None else [span[1], span[1]]

def capture_bug(alignment):
    i = 0
    found_span = False
    buggy_span = []
    
    while i < len(alignment):
        potential_buggy = {}
        start_idx = i - 1
        
        # if we see a sequence of deletes, replaces, and inserts --> this is a potential bug
        while i < len(alignment) and ('DELETE' in alignment[i][2] or 'REPLACE' in alignment[i][2] or
                                      'INSERT' in alignment[i][1]):
        
            potential_buggy[i] = alignment[i]
            i += 1
        
        if len(potential_buggy) > 1:

            # save the start and end anchors and the sequence of edits
            buggy_span.append(Range(start_idx, i, potential_buggy))
#             buggy_span.append(Range(start_idx, i, dict(sorted(potential_buggy.items(),
#                                                               key=lambda x: sort_range(x[1])))))
    
        elif len(potential_buggy) == 0:
            i += 1
            
    return buggy_span

In [18]:
def normalize_str(s):
    norm_s = normalize_alef_ar(s)
    norm_s = normalize_alef_maksura_ar(norm_s)
    norm_s = normalize_teh_marbuta_ar(norm_s)
    return norm_s

def clean_potential_merge(src, tgt):
    i = 0
    src_ = normalize_str(' '.join(src)).split()
    tgt_ = normalize_str(' '.join(tgt)).split()
    
    pairwise = [''.join(x) for x in list(zip(src_, src_[1:]))]
    new_src = []
    
    while i < len(pairwise):

        if pairwise[i] in tgt_:
            new_src.append(pairwise[i])
            i += 2
        else:
            new_src.append(src[i])
            i += 1
            
            
    while i < len(src):
        new_src.append(src[i])
        i += 1
    
    
    return new_src

def construct_src_tgt(bug):
    src = []
    tgt = []
    for v in bug.ops.values():

        if v[2] == 'DELETE':
            src.append(v[0])
            tgt.append('NIL')

        elif v[1] == 'INSERT':
            src.append('NIL')
            tgt.append(v[0])
            
        elif v[2] == 'REPLACE':
            src.append(v[0])
            tgt.append(v[1])
    
    return src, tgt

def consruct_clean_src_tgt(align):
    src_tokens = []
    tgt_tokens = []

    for x in align:
        if x[1] != 'INSERT':
            src_tokens.append(x[0])
            tgt_tokens.append(x[1])
        else:
            src_tokens.append('')
            tgt_tokens.append(x[0])
    
    return src_tokens, tgt_tokens

In [19]:
def perfect_align(src, tgt):
    assert len(src) == len(tgt)
    visited = set()
    basic_edit = get_edit(src, tgt) # get the basic edit between src and tgt
    best_edit = {'src': src, 'tgt': tgt, 'edit': basic_edit}

    best_src, best_tgt = src, tgt
    i = 0
    
    while i < len(src):
        
        if (src[i] == 'NIL' and tgt[i] != 'NIL'): # insert or a potential split
            visited.add((i, src[i], tgt[i]))
            prepend = is_split_merge(src, tgt, i, 'prepend', src_first=True)
            append = is_split_merge(src, tgt, i, 'append', src_first=True)
            best_edit = get_best_edit(best_edit, prepend, append)

        
        elif (src[i] != 'NIL' and tgt[i] == 'NIL'): # delete or a potential merge
            visited.add((i, src[i], tgt[i]))
            prepend = is_split_merge(tgt, src, i, 'prepend', src_first=False)
            append = is_split_merge(tgt, src, i, 'append', src_first=False)
            best_edit = get_best_edit(best_edit, prepend, append)
        
        
        # rewind the index in case of a change
        if best_edit['src'] != src or best_edit['tgt'] != tgt:
            src, tgt = best_edit['src'], best_edit['tgt']
            i = 0
            continue
        
        i += 1
        
    return best_edit['src'], best_edit['tgt']
        
def is_split_merge(src, tgt, i, mode='prepend', src_first=False):
    
    plausible = False

    if mode == 'prepend':
        j = i - 1

        while j >= 0 and tgt[j] == 'NIL': # find the latest non-NIL 
            j -= 1
        
        if j >= 0:
            src_ = src[:i] + src[i + 1:]
#             tgt_ = tgt[:j] + [tgt[j] + ' ' + tgt[i]] + tgt[i + 1:]
            tgt_ = tgt[:j] + [tgt[j] + ' ' + tgt[i]] + [tgt[x] for x in range(j+1, len((tgt))) if x != i]

            if len(src_) == len(tgt_):
                edit_all = get_edit(src_, tgt_)
                edit_no_space = get_edit(src_, [x.replace(' ','') for x in tgt_])
                
                edit = edit_no_space + 0.1 * (edit_all - edit_no_space)
            
                if src_first:
                    # split on target
                    return {'src': src_, 'tgt': tgt_,
                            'edit': edit}
                else:
                    # merge on source
                    return {'src': tgt_ , 'tgt': src_,
                            'edit': edit}
        
        
        if plausible == False:
            if src_first:
                return {'src': src, 'tgt': tgt, 'edit': get_edit(src, tgt)}
            else:
                return {'src': tgt, 'tgt': src, 'edit': get_edit(src, tgt)}
    
    elif mode == 'append': # find the first non-NIL
        j = i + 1

        while j < len(tgt) and tgt[j] == 'NIL':
            j += 1
        
        if j < len(tgt):
            src_ = src[:i] + src[i + 1:]
#             tgt_ = tgt[:i] + [tgt[i] + ' ' + tgt[j]] + tgt[j + 1:]
            tgt_ = tgt[:i] + [tgt[i] + ' ' + tgt[j]] + [tgt[x] for x in range(i+1, len((tgt))) if x != j]
            
    
            if len(src_) == len(tgt_):
                edit_all = get_edit(src_, tgt_)
                edit_no_space = get_edit(src_, [x.replace(' ','') for x in tgt_])
                
                edit = edit_no_space + 0.1 * (edit_all - edit_no_space)

                if src_first:
                    # split on target
                    return {'src': src_, 'tgt': tgt_,
                            'edit': edit}
                else:
                    # merge on source
                    return {'src': tgt_ , 'tgt': src_,
                            'edit': edit}
        
        if plausible == False:
            if src_first:
                return {'src': src, 'tgt': tgt, 'edit': get_edit(src, tgt)}
            else:
                return {'src': tgt, 'tgt': src, 'edit': get_edit(src, tgt)}

def get_best_edit(edit1, edit2, edit3):
    edits = (edit1, edit2, edit3)
    # in case of a tie, prefer the basic edit
    if edit1['edit'] == edit2['edit'] == edit3['edit']:
        return edit1
    
    edits_w_idx = [(i, x['edit']) for i, x in enumerate(edits)]
    
    min_edit = min(edits_w_idx, key=lambda x: x[1])[0]
    
    return edits[min_edit]
    
    
def get_edit(src, tgt):
    edit = 0
    
    for i in range(len(src)):
        s, t = normalize_str(src[i]), normalize_str(tgt[i])
        edit += edits(s.replace('PNX','') if s != 'NIL' else '',
                      t.replace('PNX','') if t != 'NIL' else '')
    
    return edit
    

def edits(s1, s2):
    return editdistance.distance(s1, s2)

In [20]:
def bug_fix(align, seq_bug):
    src = []
    tgt = []
    start = 0

    for bug in seq_bug:
        src_tokens, tgt_tokens = consruct_clean_src_tgt(align[start: bug.start + 1])
        src += src_tokens
        tgt += tgt_tokens
    
        p_src, p_tgt = construct_src_tgt(bug)
        _p_src, _p_tgt = perfect_align(p_src, p_tgt)
        
        src += _p_src
        tgt +=  _p_tgt
        
        start = bug.end


    src_tokens, tgt_tokens = consruct_clean_src_tgt(align[start: ])
    src += src_tokens
    tgt += tgt_tokens
    
    assert len(src) == len(tgt)
    return src, tgt

In [21]:
def post_process_alignment(alignment):
    clean_alignment = []
    for i, align in enumerate(alignment):
        seq_bug = capture_bug(align)

        if len(seq_bug) == 0:
            src, tgt = consruct_clean_src_tgt(align)
            
            clean_alignment.append({'src': src, 'tgt' : tgt})
        
        else:
            src, tgt = bug_fix(align, seq_bug)
            clean_alignment.append({'src': src, 'tgt': tgt})
    
    return clean_alignment

In [22]:
def write_data(alignment, path):
    with open(path, mode='w') as f:
        f.write('SOURCE\tTARGET')
        f.write('\n')
        for example in alignment:
            for s, t in zip(example['src'], example['tgt']):
                s = s.replace('PNX', '').replace('NIL', '')
                t = t.replace('PNX', '').replace('NIL', '')

                f.write(f'{s}\t{t}')
                f.write('\n')
            f.write('\n')

In [23]:
# basic_alignment = read_alignment('/scratch/ba63/gec/data/ZAEBUC-v1.0/data/'\
#                                  'ar/dev/dev.alignment.pnx')
# src = read_data('/scratch/ba63/gec/data/ZAEBUC-v1.0/data/ar/dev/dev.sent.raw.pnx.tok.ced')
# cor = read_data('/scratch/ba63/gec/data/ZAEBUC-v1.0/data/ar/dev/dev.sent.cor.pnx.tok.ced')

# clean_alignment = post_process_alignment(basic_alignment)
# write_data(clean_alignment, 'bashar_alignment_zaebuc.txt')


In [24]:
basic_alignment = read_alignment('/scratch/ba63/gec/data/QALB-0.9.1-Dec03-2021-SharedTasks/'\
                                 'data/2014/tune/tune.alignment.pnx')
src = read_data('/scratch/ba63/gec/data/QALB-0.9.1-Dec03-2021-SharedTasks/data/'\
                '2014/tune/QALB-2014-L1-Tune.sent.no_ids.clean.ced')
cor = read_data('/scratch/ba63/gec/data/QALB-0.9.1-Dec03-2021-SharedTasks/data/'\
                '2014/tune/QALB-2014-L1-Tune.cor.no_ids.ced')

# clean_alignment = post_process_alignment(basic_alignment)
# write_data(clean_alignment, 'bashar_alignment_qalb14.txt') # 741, 2, 780


In [25]:
# clean_alignment = post_process_alignment(basic_alignment)
# write_data(clean_alignment, 'bashar_alignment_qalb14.txt')


In [26]:
basic_alignment[741]

[['هل', 'هل', 'KEEP', (1, 1)],
 ['يوجد', 'يوجد', 'KEEP', (2, 2)],
 ['سفاح', 'سفاح', 'KEEP', (3, 3)],
 ['ومجرم', 'ومجرم', 'KEEP', (4, 4)],
 ['بهذا', 'بهذا', 'KEEP', (5, 5)],
 ['الشكل', 'الشكل', 'KEEP', (6, 6)],
 ['في', 'في', 'KEEP', (7, 7)],
 ['العالم', 'العالم', 'KEEP', (8, 8)],
 ['الان', 'الآن', 'REPLACE', (9, 9)],
 ['PNX؟', 'PNX؟', 'KEEP', (10, 10)],
 ['لا', 'لا', 'KEEP', (11, 11)],
 ['يتوانون', 'يتوانون', 'KEEP', (12, 12)],
 ['عن', 'عن', 'KEEP', (13, 13)],
 ['قصف', 'قصف', 'KEEP', (14, 14)],
 ['المدنيين', 'المدنيين', 'KEEP', (15, 15)],
 ['بالمدافع', 'بالمدافع', 'KEEP', (16, 16)],
 ['PNX،', 'INSERT', (None, 17)],
 ['والدبابات', 'والدبابات', 'KEEP', (17, 18)],
 ['PNX،', 'INSERT', (None, 19)],
 ['والزوارق', 'والزوارق', 'KEEP', (18, 20)],
 ['الحربية', 'الحربية', 'KEEP', (19, 21)],
 ['PNX،', 'PNX.', 'REPLACE', (20, 22)],
 ['الى', 'إلى', 'REPLACE', (21, 23)],
 ['متى', 'متى', 'KEEP', (22, 24)],
 ['سيبقى', 'سيبقى', 'KEEP', (23, 25)],
 ['العالم', 'العالم', 'KEEP', (24, 26)],
 ['يعطي', 'يعطي',

In [28]:
bug_ex = capture_bug(basic_alignment[2])
for bug in bug_ex:
    print(bug)

    src, tgt = construct_src_tgt(bug)
    print(src)
    print(tgt)
    print()
#     print(f'Basic edit: {get_basic_edit(src, tgt)}')
    print('========')
#     import pdb; pdb.set_trace()
    new_src, new_tgt = perfect_align(src, tgt)
    
    print(new_src)
    print(new_tgt)
    print('========')
    print()

{'start': 11, 'end': 15, 'ops': {12: ['محكمه', 'محكمة', 'REPLACE', (13, 13)], 13: ['PNX،', 'INSERT', (None, 14)], 14: ['فهوا', 'فهو', 'REPLACE', (14, 15)]}}
['محكمه', 'NIL', 'فهوا']
['محكمة', 'PNX،', 'فهو']

['محكمه', 'NIL', 'فهوا']
['محكمة', 'PNX،', 'فهو']

{'start': 15, 'end': 18, 'ops': {16: ['PNX.', 'INSERT', (None, 17)], 17: ['ان', 'إن', 'REPLACE', (16, 18)]}}
['NIL', 'ان']
['PNX.', 'إن']

['NIL', 'ان']
['PNX.', 'إن']

{'start': 22, 'end': 25, 'ops': {23: ['PNX،', 'INSERT', (None, 24)], 24: ['واقول', 'وأقول', 'REPLACE', (22, 25)]}}
['NIL', 'واقول']
['PNX،', 'وأقول']

['NIL', 'واقول']
['PNX،', 'وأقول']

{'start': 27, 'end': 30, 'ops': {28: ['PNX،', 'INSERT', (None, 29)], 29: ['او', 'أو', 'REPLACE', (26, 30)]}}
['NIL', 'او']
['PNX،', 'أو']

['NIL', 'او']
['PNX،', 'أو']

{'start': 31, 'end': 34, 'ops': {32: ['PNX،', 'INSERT', (None, 33)], 33: ['ان', 'أن', 'REPLACE', (29, 34)]}}
['NIL', 'ان']
['PNX،', 'أن']

['NIL', 'ان']
['PNX،', 'أن']

{'start': 34, 'end': 38, 'ops': {35: ['وطغمتهو'